In [1]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import lightgbm as lgb
import math
from sklearn.metrics import mean_squared_error as mse
from sklearn.linear_model import LinearRegression
import warnings
import pdb
warnings.filterwarnings('ignore')
import random
from tqdm import tqdm
from sklearn.model_selection import StratifiedKFold, KFold

In [2]:
def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)  # 禁止hash随机化

set_seed(42)


In [3]:
# label: '推料器自动指令'
# train_data_path = './data/train/'
# test_data_path = './data/test/'
#
# train_cols = ['推料器自动指令', 'CO含量', 'HCL含量', 'NOx含量', 'SO2含量', '二次风调门', '二次风量', '给水流量',
#         '炉排实际运行指令', '炉排自动投退信号', '汽包水位', '推料器启停', '推料器自动投退信号',
#         '氧量设定值', '一次风调门', '一次风量', '引风机转速', '主蒸汽流量']
#
# test_cols = ['CO含量', 'HCL含量', 'NOx含量', 'SO2含量', '二次风调门', '二次风量', '给水流量',
#         '炉排实际运行指令', '炉排自动投退信号', '汽包水位', '推料器启停', '推料器自动投退信号',
#         '氧量设定值', '一次风调门', '一次风量', '引风机转速', '主蒸汽流量']
#
# print(len(train_cols))
# print(len(test_cols))

In [4]:
# train_df = pd.DataFrame()
# train_df['时间'] = pd.read_csv(train_data_path+'CO含量.csv')['时间']
#
# for n in tqdm(train_cols):
#     train_df[n] = pd.read_csv(train_data_path+f'{n}.csv')[n]
#
#

In [5]:
# test_df = pd.DataFrame()
# test_df['时间'] = pd.read_csv(test_data_path+'CO含量.csv')['时间']
#
# for n in tqdm(test_cols):
#     test_df[n] = pd.read_csv(test_data_path+f'{n}.csv')[n]
#
#

In [6]:
# # 有重复的行，需要去重
# print('训练集中重复的行数：', train_df.duplicated().sum())
# train_df = train_df.drop_duplicates()
#
# print('测试集中重复的行数：', test_df.duplicated().sum())
# test_df = test_df.drop_duplicates()
#
#

In [7]:
# train_df.to_csv('./data/train.csv', index=False)
# test_df.to_csv('./data/test.csv', index=False)


In [8]:
train_df = pd.read_csv('./data/train.csv')
train_df = train_df.dropna()

test_df = pd.read_csv('./data/test.csv')
train_df.info()
test_df.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 141188 entries, 0 to 141187
Data columns (total 19 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   时间         141188 non-null  object 
 1   推料器自动指令    141188 non-null  float64
 2   CO含量       141188 non-null  float64
 3   HCL含量      141188 non-null  float64
 4   NOx含量      141188 non-null  float64
 5   SO2含量      141188 non-null  float64
 6   二次风调门      141188 non-null  float64
 7   二次风量       141188 non-null  float64
 8   给水流量       141188 non-null  float64
 9   炉排实际运行指令   141188 non-null  float64
 10  炉排自动投退信号   141188 non-null  object 
 11  汽包水位       141188 non-null  float64
 12  推料器启停      141188 non-null  object 
 13  推料器自动投退信号  141188 non-null  object 
 14  氧量设定值      141188 non-null  float64
 15  一次风调门      141188 non-null  float64
 16  一次风量       141188 non-null  float64
 17  引风机转速      141188 non-null  float64
 18  主蒸汽流量      141188 non-null  float64
dtypes: float64(15), object(

In [9]:
data = pd.concat([train_df, test_df])

data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 142988 entries, 0 to 1799
Data columns (total 19 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   时间         142988 non-null  object 
 1   推料器自动指令    141188 non-null  float64
 2   CO含量       142988 non-null  float64
 3   HCL含量      142988 non-null  float64
 4   NOx含量      142988 non-null  float64
 5   SO2含量      142988 non-null  float64
 6   二次风调门      142988 non-null  float64
 7   二次风量       142988 non-null  float64
 8   给水流量       142988 non-null  float64
 9   炉排实际运行指令   142988 non-null  float64
 10  炉排自动投退信号   142988 non-null  object 
 11  汽包水位       142988 non-null  float64
 12  推料器启停      142988 non-null  object 
 13  推料器自动投退信号  142988 non-null  object 
 14  氧量设定值      142988 non-null  float64
 15  一次风调门      142988 non-null  float64
 16  一次风量       142988 non-null  float64
 17  引风机转速      142988 non-null  float64
 18  主蒸汽流量      142988 non-null  float64
dtypes: float64(15), object(4)

In [10]:
data['推料器启停'].value_counts()

True     102060
False     40928
Name: 推料器启停, dtype: int64

In [11]:
data = data[data['推料器启停']==1]
# data.info()

In [12]:
data = data.drop(columns=['推料器启停'])
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 102060 entries, 169 to 1799
Data columns (total 18 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   时间         102060 non-null  object 
 1   推料器自动指令    100260 non-null  float64
 2   CO含量       102060 non-null  float64
 3   HCL含量      102060 non-null  float64
 4   NOx含量      102060 non-null  float64
 5   SO2含量      102060 non-null  float64
 6   二次风调门      102060 non-null  float64
 7   二次风量       102060 non-null  float64
 8   给水流量       102060 non-null  float64
 9   炉排实际运行指令   102060 non-null  float64
 10  炉排自动投退信号   102060 non-null  object 
 11  汽包水位       102060 non-null  float64
 12  推料器自动投退信号  102060 non-null  object 
 13  氧量设定值      102060 non-null  float64
 14  一次风调门      102060 non-null  float64
 15  一次风量       102060 non-null  float64
 16  引风机转速      102060 non-null  float64
 17  主蒸汽流量      102060 non-null  float64
dtypes: float64(15), object(3)
memory usage: 14.8+ MB


In [13]:
data['gas'] = data['CO含量'] + data['HCL含量'] + data['NOx含量'] + data['SO2含量']
# feat_list = ['二次风调门', '二次风量', '给水流量', '炉排实际运行指令', '汽包水位', '氧量设定值', '一次风调门', '一次风量', '引风机转速', '主蒸汽流量']
# ['给水流量', '氧量设定值', '一次风量', '主蒸汽流量', '汽包水位']
feat_list = ['炉排实际运行指令','gas','CO含量','HCL含量','NOx含量','SO2含量','二次风调门', '二次风量', '给水流量', '汽包水位', '氧量设定值', '一次风调门', '一次风量', '引风机转速', '主蒸汽流量']
for f in tqdm(feat_list):
    shift_f = []
    shift_lf = []
    shift_d = []
    for i in range(200):
        colname = f + '_shift_{}'.format(i + 1)
        data[colname] = data[f].shift(i + 1)
        shift_f.append(colname)

        colname = f + '_lshift_{}'.format(-(i + 1))
        data[colname] = data[f].shift(-(i + 1))
        shift_lf.append(colname)
 
        # .diff用于计算一列中某元素与该列中前？个元素的差值（默认前一个元素）
        # colname = f + '_diff_{}'.format(i + 1)
        # data[colname] = data[f].diff(i + 1)
        # colname = f + '_diff_{}'.format(-(i + 1))
        # data[colname] = data[f].diff(-(i + 1))

    # data[f+'_diff'] = data[f].diff(1)
    # 对每一行：shift_列的值取平均，即将前n天的值取平均，取最大值，最最小值，最标准差

    data[f + '_fore_steps_mean'] = data[shift_f].mean(1)
    data[f + '_fore_steps_max'] = data[shift_f].max(1)
    data[f + '_fore_steps_min'] = data[shift_f].min(1)
    data[f + '_fore_steps_std'] = data[shift_f].std(1)
    data[f + '_fore_steps_std'] = data[shift_f].skew(1)

    data[f + '_lfore_steps_mean'] = data[shift_lf].mean(1)
    data[f + '_lfore_steps_max'] = data[shift_lf].max(1)
    data[f + '_lfore_steps_min'] = data[shift_lf].min(1)
    data[f + '_lfore_steps_std'] = data[shift_lf].std(1)
    data[f + '_lfore_steps_std'] = data[shift_lf].skew(1)

    data.drop(shift_f, axis=1, inplace=True)
    data.drop(shift_lf, axis=1, inplace=True)

100%|██████████| 15/15 [01:17<00:00,  5.19s/it]


In [14]:
#
data['炉排自动投退信号'] = data['炉排自动投退信号'].map(int)
# data['推料器启停'] = data['推料器启停'].map(int)
data['推料器自动投退信号'] = data['推料器自动投退信号'].map(int)

In [15]:
import featuretools as ft

In [16]:
# from sklearn.datasets import load_iris
# import pandas as pd
# import featuretools as ft
#
# # Load data and put into dataframe
# iris = load_iris()
# df = pd.DataFrame(iris.data, columns = iris.feature_names)
# df['species'] = iris.target
# df['species'] = df['species'].map({0: 'setosa', 1: 'versicolor', 2: 'virginica'})

In [17]:
# # Make an entityset and add the entity
# es = ft.EntitySet(id = 'iris')
# es.add_dataframe(dataframe_name = 'data', dataframe = df,
#                          make_index = True, index = 'index')
#
# # Run deep feature synthesis with transformation primitives
# feature_matrix, feature_defs = ft.dfs(entityset = es, target_dataframe_name = 'data',
#                                       trans_primitives = ['add_numeric', 'multiply_numeric'])
#
# feature_matrix.head()


In [18]:
# groupby + shift(差值特征) + transform(count, mean, max, min, skew)

# def get_shift_feats(data, gap_list=[1], gp_col=[], target_col=''):
#     # gp_col可以是个列表进行多次分组，e.g. gp_col = [id1, id2]
#     for gap in gap_list:
#         # 后面减前面
#         data[''.join(gp_col)+f'{target_col}next{gap}'] = data.groupby(gp_col)[target_col].shift(-gap)
#         data[''.join(gp_col)+f'{target_col}next{gap}'] = data[''.join(gp_col)+f'{target_col}next{gap}'] - data[target_col]
# 
#         # 前面减后面
#         data[''.join(gp_col)+f'{target_col}prev{gap}'] = data.groupby(gp_col)[target_col].shift(+gap)
#         data[''.join(gp_col)+f'{target_col}prev{gap}'] = data[''.join(gp_col)+f'{target_col}next{gap}'] - data[target_col]
# 
# 
#         # 统计其不为nan的值
#         data[''.join(gp_col)+f'{target_col}next{gap}count'] = data.groupby(gp_col)[''.join(gp_col)+f'{target_col}next{gap}'].transform('count')
#         data[''.join(gp_col)+f'{target_col}prev{gap}count'] = data.groupby(gp_col)[''.join(gp_col)+f'{target_col}prev{gap}'].transform('count')
# 
#         # 统计其平均值
#         data[''.join(gp_col)+f'{target_col}next{gap}mean'] = data.groupby(gp_col)[''.join(gp_col)+f'{target_col}next{gap}'].transform('mean')
#         data[''.join(gp_col)+f'{target_col}prev{gap}mean'] = data.groupby(gp_col)[''.join(gp_col)+f'{target_col}prev{gap}'].transform('mean')
# 
#         # 统计其最大值
#         data[''.join(gp_col)+f'{target_col}next{gap}max'] = data.groupby(gp_col)[''.join(gp_col)+f'{target_col}next{gap}'].transform('max')
#         data[''.join(gp_col)+f'{target_col}prev{gap}max'] = data.groupby(gp_col)[''.join(gp_col)+f'{target_col}prev{gap}'].transform('max')
# 
#         # 统计其最小值
#         data[''.join(gp_col)+f'{target_col}next{gap}min'] = data.groupby(gp_col)[''.join(gp_col)+f'{target_col}next{gap}'].transform('min')
#         data[''.join(gp_col)+f'{target_col}prev{gap}min'] = data.groupby(gp_col)[''.join(gp_col)+f'{target_col}prev{gap}'].transform('min')
# 
#         # 统计其skew值
#         data[''.join(gp_col)+f'{target_col}next{gap}skew'] = data.groupby(gp_col)[''.join(gp_col)+f'{target_col}next{gap}'].transform('skew')
#         data[''.join(gp_col)+f'{target_col}prev{gap}skew'] = data.groupby(gp_col)[''.join(gp_col)+f'{target_col}prev{gap}'].transform('skew')
# 
#     return data
# 

In [19]:

# gp_tg_cols = [(['推料器启停'], '给水流量'),
#               (['推料器启停'], '主蒸汽流量')]
# 
# 

In [20]:
# for col in tqdm(gp_tg_cols):
#     data = get_shift_feats(data, gap_list=list(range(1, 10)), gp_col=col[0], target_col=col[1])

In [21]:
# data.columns = [str(i) for i in data.columns]
# for i in data.columns:
#     print(i)

In [22]:
# # 内存压缩
def reduce_mem_usage(df, verbose=True):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024 ** 2
    for col in tqdm(df.columns):
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
    end_mem = df.memory_usage().sum() / 1024 ** 2
    if verbose: print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (
                start_mem - end_mem) / start_mem))
    return df

# 压缩使用内存
# 由于数据比较大，所以合理的压缩内存节省空间尤为的重要
# 使用reduce_mem_usage函数可以压缩近70%的内存占有。
data = reduce_mem_usage(data)

100%|██████████| 259/259 [00:07<00:00, 34.87it/s]


Mem. usage decreased to 53.34 Mb (73.7% reduction)


In [23]:
print('nums features: ', len(data.columns))

nums features:  259


In [24]:
label = '推料器自动指令'
test_data = data.tail(1800)
train = data.iloc[:-1800, :]
train.reset_index(drop=True, inplace=True)
features = train.columns.drop(['时间', label]).tolist()
train_x, train_y = train[features], train[label]

In [25]:
# train

In [26]:
# 交叉验证所使用的第三方库


kf = KFold(n_splits=5, shuffle=True)
test_x = test_data[features]
test_lgb = np.zeros(test_x.shape[0])



for i, (train_index, valid_index) in enumerate(kf.split(train_x, train_y)):
    print('************************************  {} fold************************************'.format(str(i + 1)))
    trn_x, trn_y, val_x, val_y = train_x.iloc[train_index], train_y[train_index], \
                                 train_x.iloc[valid_index], train_y[valid_index]

    dtrain = lgb.Dataset(trn_x, label=trn_y)
    dvalid = lgb.Dataset(val_x, label=val_y)

    watchlist = [dtrain, dvalid]

    params = {
        'boosting_type': 'gbdt',
        'objective': 'regression_l1',
        'metric': 'l2_root',
        'learning_rate': 0.01,
        'reg_alpha': 0.7,
        'reg_lambda': 35,
        'bagging_fraction': 0.7,
        'bagging_freq': 5,
        'feature_fraction': 0.7,
        "random_seed": 1,
    }
    model = lgb.train(params, train_set=dtrain, num_boost_round=10000, early_stopping_rounds=1000, valid_sets=watchlist,
                      verbose_eval=200)




    test_pred = model.predict(test_x)
    test_lgb += test_pred / kf.n_splits


    print("Features importance...")
    gain = model.feature_importance('gain')
    feat_imp = pd.DataFrame({'feature': model.feature_name(),
                             'split': model.feature_importance('split'),
                             'gain': 100 * gain / gain.sum()}).sort_values('gain', ascending=False)
    print('Top 50 features:\n', feat_imp.head(50))

************************************  1 fold************************************
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.033331 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 60197
[LightGBM] [Info] Number of data points in the train set: 80208, number of used features: 255
[LightGBM] [Info] Start training from score 70.000000
Training until validation scores don't improve for 1000 rounds
[200]	training's rmse: 4.3598	valid_1's rmse: 4.33038
[400]	training's rmse: 3.42659	valid_1's rmse: 3.39811
[600]	training's rmse: 3.16131	valid_1's rmse: 3.12643
[800]	training's rmse: 2.6728	valid_1's rmse: 2.63813
[1000]	training's rmse: 2.24504	valid_1's rmse: 2.20423
[1200]	training's rmse: 2.09262	valid_1's rmse: 2.04213
[1400]	training's rmse: 1.9601	valid_1's rmse: 1.90752
[1600]	training's rmse: 1.84742	valid_1's rmse: 1.79612
[1800]	training's rmse: 1.768	valid_1's rmse: 1.71699
[2000]	traini

KeyboardInterrupt: 

In [ ]:
pd.Series(test_lgb).describe()

In [ ]:

# sub = test_data[['时间','推料器启停']]
# sub['lgb' + label] = test_lgb
# sub.reset_index(inplace=True)
# sub.columns = ['ID', 'Time', 'Aps', 'Ai']



sub = pd.DataFrame()
sub['ID'] = list(range(1,1801))
sub['Time'] = test_df['时间']
sub['Aps'] = test_df['推料器启停'].astype(bool)
sub['Ai'] = test_lgb

sub.to_csv('./result_lgb.csv', index=False)

